# Fine-tune the model for object detection

Note: the custom dataset must be in COCO form, i.e., the labels are saved in a json file

## 1 Calculating the number of iterations in Detectron2

Note: There is no concept of epoch in detectron2, so we need to calculate the number of iterations

In [ ]:
NUM_GPUS = 1 
IMS_PER_BATCH = 4 # Batch_size 
# TOTAL_NUM_IMAGES = 1243 # images in training dataset
Epochs = 100

# MAX_ITER = (Epochs * TOTAL_NUM_IMAGES) / (NUM_GPUS * IMS_PER_BATCH)
# print("The number of itearations is: ", MAX_ITER)


print("The number of itearations in GJO_train_2628 is: ", (Epochs * 2628) / (NUM_GPUS * IMS_PER_BATCH))
print("The number of itearations in GJO_train_2076 is: ", (Epochs * 2076) / (NUM_GPUS * IMS_PER_BATCH))
print("The number of itearations in GJO_train_1594 is: ", (Epochs * 1594) / (NUM_GPUS * IMS_PER_BATCH))
print("The number of itearations in GJO_train_1013 is: ", (Epochs * 1013) / (NUM_GPUS * IMS_PER_BATCH))
print("The number of itearations in GJO_train_527 is: ", (Epochs * 527) / (NUM_GPUS * IMS_PER_BATCH))
print("The number of itearations in GJO_train_282 is: ", (Epochs * 282) / (NUM_GPUS * IMS_PER_BATCH))
print("The number of itearations in GJO_train_124 is: ", (Epochs * 124) / (NUM_GPUS * IMS_PER_BATCH))

In [1]:
NUM_GPUS = 1 
IMS_PER_BATCH = 4 # Batch_size 
# TOTAL_NUM_IMAGES = 1243 # images in training dataset
Epochs = 100

# MAX_ITER = (Epochs * TOTAL_NUM_IMAGES) / (NUM_GPUS * IMS_PER_BATCH)
# print("The number of itearations is: ", MAX_ITER)

print("The number of itearations in GJO_train_2628 is: ", (Epochs * 108) / (NUM_GPUS * IMS_PER_BATCH))

The number of itearations in GJO_train_2628 is:  2700.0


## 3 Register fine-tuning dataset

Register the dataset in the **tools/object_detection_benchmark_1.py** file, using the following example code

In [ ]:
# register my custom dataset, if your dataset is in COCO format:
       
# Flux
# register_coco_instances("Flux_train", {}, "/scratch/tjian/Data/Flux/labeled_images/annotations/train.json", "/scratch/tjian/Data/Flux/labeled_images/train/")
# register_coco_instances("Flux_val", {}, "/scratch/tjian/Data/Flux/labeled_images/annotations/val.json", "/scratch/tjian/Data/Flux/labeled_images/val/")


# 0609_1_round_SF
register_coco_instances("0609_1_round_train", {}, "/scratch/tjian/Data/Flux/TUD_Vietnam/0609_1_round_SF/annotations/train.json", "/scratch/tjian/Data/Flux/TUD_Vietnam/0609_1_round_SF/train/")
register_coco_instances("0609_1_round_val", {}, "/scratch/tjian/Data/Flux/TUD_Vietnam/0609_1_round_SF/annotations/val.json", "/scratch/tjian/Data/Flux/TUD_Vietnam/0609_1_round_SF/val/")


## 4 Define the metric 

Save the best model weights with a type of metric during validatingduring validating in the **tools/object_detection_benchmark_1.py** file, using the following example code

In [ ]:
# Choose the metric to validate the model
metric_name="bbox/AP50"
# metric_name="bbox/AP"
# metric_name="bbox/AP75"
# metric_name="bbox/APs"
# metric_name="bbox/APm"
# metric_name="bbox/APl"

## 5 Fine-tune the model

Using a Semi-supervised learning (5.1) or a supervised learning (5.2) method

Note: Freeze the first 2 layers of ResNet: *MODEL.BACKBONE.FREEZE_AT 2*; Freeze the first 4 layers of ResNet: *MODEL.BACKBONE.FREEZE_AT 4*

### 5.1 Semi-supervised learning

Steps:

(1) Load the ResNet weights pre-trained by self-supervised learning method (SimClR or SwAV), and generate the Faster R-CNN based on this backbone;

(2) Modify the Faster R-CNN hyperparameters in "/configs/config/benchmark/object_detection/COCOInstance/R50_C4_COCO/tiles_224/XXX.yaml" file, especially the **dataset name**.

(3) Fine-tune the FasterRCNN in a supervised learning method.

Notice:

If gpus=1, no.images=200, batch_size=4, and epochs=100, then "SOLVER.MAX_ITER 5000"  # 5000=(200/4)*100

#### Self_supervised method: SwAV

Note: you can modify **MODEL.BACKBONE.FREEZE_AT 2 (or 4)** parameter to define the first 2 (or 4) layers are frozen when fine-tuning

##### (1) Faster R-CNN with **ResNet50**

In [ ]:
# Self_supervised method: SwAV

# Moded is trained on image tiles 224*224
# gpu=1
# num_epochs=100
# batch_size=4
# Fine-tune: F4

# Pre-trian: RN50_300K_100e
!python tools/object_detection_benchmark_1.py \
    --config-file /scratch/tjian/PythonProject/deep_plastic_Flux_SSL/configs/config/benchmark/object_detection/COCOInstance/R50_C4_COCO/tiles_224/Sliced_Fine/FRC_50.yaml \
     --num-gpus 1 SOLVER.MAX_ITER 2700 TEST.EVAL_PERIOD 27 SOLVER.IMS_PER_BATCH 4 MODEL.BACKBONE.FREEZE_AT 0 MODEL.WEIGHTS /scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/RN50_300K_100e/SSL_F4/model_best_86.1242.pth OUTPUT_DIR /scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/0609_1_round_SF/



##### (2) Faster R-CNN with **ResNet101**

In [ ]:
# Moded is trained on image tiles 224*224
# gpu=1
# num_epochs=100
# batch_size=4

# Fine-tune: F4

# Pre-trian: RN101_100K_100e
!python tools/object_detection_benchmark_1.py \
    --config-file /scratch/tjian/PythonProject/deep_plastic_Flux_SSL/configs/config/benchmark/object_detection/COCOInstance/R50_C4_COCO/tiles_224/Sliced_Fine/FRC_101.yaml \
     --num-gpus 1 SOLVER.MAX_ITER 2700 TEST.EVAL_PERIOD 27 SOLVER.IMS_PER_BATCH 4 MODEL.BACKBONE.FREEZE_AT 4 MODEL.WEIGHTS XXX OUTPUT_DIR XXXX

